<a href="https://colab.research.google.com/github/michealman114/Natural-Language-Models-for-Hate-Speech-Classification/blob/main/TestingLSTMsForHateSpeech.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Here's most of our code for lazy testing different approaches. We still need to add in the k-fold cross validation functions from the original experiments and add in the title context functionality.


To use this notebook you will need
- to be able to mount your google drive in the 4th code cell (with the folder containing the folder of stored tensors in your home directory)
- to upload CommentsDatasets.py, models.py, TrainingLoops.py in the left sidebar file manager

In [1]:
import torch
import torch.nn as nn 
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as torch_data

In [2]:
from torch import cuda

if cuda.is_available():
    device = 'cuda'
    seed = 4814
    torch.cuda.manual_seed_all(seed)
    print("running on GPU:", torch.cuda.get_device_name(0))
else:
    print("running on CPU")

running on GPU: Tesla T4


In [3]:
from CommentsDatasets import * # torch dataset setup
from models import * # all our LSTM based models
from TrainingLoops import * # our training and k-fold cross validation methods

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
!ls '/content/drive/MyDrive/Natural Language Models for Hate Speech Classification'

'Classification with BERT.ipynb'   StoredTensors
'Embeddings from BERT.ipynb'	   TestingLSTMsForHateSpeech.ipynb
'LSTM Full Model with Attention'


Note to self (and anybody else who wants to try this): The two cells below correspond to loading data for pregenerated Word2Vec (embed_dim = 768) and BERT (embed_dim = 768) embeddings respectively. Pick the one that you want, and don't bother running the other.

Word2Vec Embeddings Results (from a lazy training loop)
- base model: Loss: 3.535007230937481
- bidirectional LSTM: Loss: 1.8555335476994514
- bidirectional LSTM with Attention: 0.5990170016884804

**Word2Vec Embeddings 10-fold cross validation (30 epochs)**
- base model:
    - Accuracy: 0.7289473684210527
    - Precision, Recall, F1: (0.5273311897106109, 0.3822843822843823, 0.4432432432432432, None)
- bidirectional model:
    - Accuracy: 0.7263157894736842
    - Precision, Recall, F1: (0.5226480836236934, 0.34965034965034963, 0.41899441340782123, None)
- bidirectional model with attention:
    - Accuracy: 0.7157894736842105
    - Precision, Recall, F1: (0.49584487534626037, 0.4172494172494173, 0.4531645569620253, None)

**Word2Vec Embeddings 10-fold cross validation (40 epochs)**
- base model:
    - Accuracy: 0.7335526315789473
    - Precision, Recall, F1: (0.5298507462686567, 0.4965034965034965, 0.5126353790613718, None)

- bidirectional model:
    - Accuracy: 0.7342105263157894
    - Precision, Recall, F1: (0.5319693094629157, 0.48484848484848486, 0.5073170731707317, None)


- bidirectional model with attention:
    - Accuracy: 0.7217105263157895
    - Precision, Recall, F1: (0.5086206896551724, 0.4125874125874126, 0.4555984555984556, None)

In [5]:
#Word2Vec 
embed_dim = 300

train_comments = torch.load('/content/drive/MyDrive/Natural Language Models for Hate Speech Classification/StoredTensors/w2v_train_comments_embeddings.pt')
train_labels = torch.load('/content/drive/MyDrive/Natural Language Models for Hate Speech Classification/StoredTensors/w2v_train_labels.pt')
train_comments_padding_masks = torch.load('/content/drive/MyDrive/Natural Language Models for Hate Speech Classification/StoredTensors/w2v_train_comments_padding_masks.pt')

In [6]:
"""
quick sanity checks:
input data should be: (batch_size, max_length, embed_dim)
labels should be: (batch_size,)
"""
print(train_comments.shape, train_comments_padding_masks.shape, train_labels.shape)

torch.Size([1528, 244, 300]) torch.Size([1528, 244]) torch.Size([1528])


The following cells correspond to loading data for pregenerated BERT (embed_dim = 300) embeddings.

BERT Embeddings Results (from a lazy training loop)
- base model: Loss: 858.4895858764648
- bidirectional LSTM: Loss: 341.25000190734863
- bidirectional LSTM with Attention: Loss: 0.0014144671586109325

These results are both intuitive and surprising. It is amsuingly surprising that the LSTMS without attention are as ludicrously terrible as they are, but it kind of makes sense. 

The enormous performance jump suggests that the attention mechanism (which in this application is just a very simple set of FC layers) is doing most of the work (even without being attention masked - which is something we need to fix pretty urgently). 

I'd be willing to bet that when working on BERT embeddings we can just trivially slap on a couple linear layers on top and get really good performance The suspicion here is that BERT more or less makes the LSTM obsolete - ironic.

Also, when we were cleaning up the dataset before feeding into word2vec we originally did some classical stuff (removing stopwords, punctuation etc) that removes important embedding context - especailly since some stopwords like "no","never","not" substantially change the meaning of a sentence - which probably also explains a good amount of why BERT performs so much better. Fixing this really obvious data processing mistake isn't too difficult, but we can do that later.

**BERT Embeddings 10-fold cross validation**
- 30 epochs: bidirectional model with attention:
    - Accuracy: 0.7532894736842105
    - Precision, Recall, F1: (0.5875370919881305, 0.45622119815668205, 0.5136186770428015, None)
- 15 epochs: bidirectional model with attention:
    - Accuracy: 0.7526315789473684
    - Precision, Recall, F1: (0.5759162303664922, 0.5069124423963134, 0.5392156862745099, None)
- 10 epochs: bidirectional model with attention:
    - Accuracy: 0.75
    - Precision, Recall, F1: (0.5658536585365853, 0.5345622119815668, 0.5497630331753554, None)
- 5 epochs: bidirectional model with attention:
    - Accuracy: 0.7381578947368421
    - Precision, Recall, F1: (0.5481283422459893, 0.47235023041474655, 0.5074257425742573, None)


Training on BERT embeddings works best with 10 epochs (15 epochs isn't bad either, but 10 just slightly outperforms it). Yay for no more overfitting.

In [4]:
embed_dim = 768

train_comments = torch.load('drive/MyDrive/Natural Language Models for Hate Speech Classification/StoredTensors/BERT_train_comments_embeddings.pt')
train_comments_padding_masks = torch.load('drive/MyDrive/Natural Language Models for Hate Speech Classification/StoredTensors/BERT_train_comments_attention_masks.pt')

train_labels = torch.load('drive/MyDrive/Natural Language Models for Hate Speech Classification/StoredTensors/BERT_train_labels.pt')

In [5]:
"""
quick sanity checks:
input data should be: (batch_size, max_length, embed_dim)
labels should be: (batch_size,)
"""
print(train_comments.shape, train_comments_padding_masks.shape, train_labels.shape)

torch.Size([1528, 512, 768]) torch.Size([1528, 512]) torch.Size([1528])


Now for a simple training loop experiment:

In [11]:
num_epochs = 30 #running on colab, use 30-40ish, see what works

In [ ]:
trained_model,training_losses = train(training_data, num_epochs, 128, Full_LSTM_Model, embed_dim = embed_dim, bidi = False, attention = False)

In [ ]:
trained_model,training_losses = train(training_data, num_epochs, 128, Full_LSTM_Model, embed_dim = embed_dim, bidi = True, attention = False)

In [ ]:
trained_model,training_losses = train(training_data, num_epochs, 128, Full_LSTM_Model, embed_dim = embed_dim, bidi = True, attention = True)



Now that we've got training working, we also set up a nice: k-fold evaluation loop is as follows:

Starting with some Word2Vec stuff (it also works for BERT but its pointless to run the base and non attention models on BERT embeddings):

In [ ]:
kfold_crossvalidation(10, train_comments, train_comments_padding_masks, train_labels, Full_LSTM_Model, device, n_epochs = 30, embed_dim = embed_dim)

In [ ]:
kfold_crossvalidation(10, train_comments, train_comments_padding_masks, train_labels, Full_LSTM_Model, device, n_epochs = 30, embed_dim = embed_dim, bidi = True)

In [ ]:
kfold_crossvalidation(10, train_comments, train_comments_padding_masks, train_labels, Full_LSTM_Model, device, n_epochs = 15, embed_dim = embed_dim, bidi = True, attention = True)

This Stuff I used for BERT evaluation


In [ ]:
kfold_crossvalidation(10, train_comments, train_comments_padding_masks, train_labels, Full_LSTM_Model, device, n_epochs = 15, embed_dim = embed_dim, bidi = True, attention = True, intermediate_checks = True)

In [6]:
kfold_crossvalidation(10, train_comments, train_comments_padding_masks, train_labels, Full_LSTM_Model, device, n_epochs = 5, embed_dim = embed_dim, bidi = True, attention = True)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))




running split 1:

LSTM with bidirectional = True, attention = True


 20%|██        | 1/5 [00:01<00:07,  1.94s/it]

Loss: 6.741696655750275


 40%|████      | 2/5 [00:04<00:06,  2.16s/it]

...

Loss: 5.838353395462036


 80%|████████  | 4/5 [00:06<00:01,  1.50s/it]

Loss: 5.234912604093552


100%|██████████| 5/5 [00:07<00:00,  1.50s/it]

Loss: 4.620144546031952
0.5495288968086243
Final Evaluation for model 10
Accuracy: 0.75
Precision, Recall, F1: (0.5666666666666667, 0.40476190476190477, 0.4722222222222222, None)

===Aggregate Stats===
Accuracy: 0.7381578947368421
Precision, Recall, F1: (0.5481283422459893, 0.47235023041474655, 0.5074257425742573, None)


In [7]:
kfold_crossvalidation(10, train_comments, train_comments_padding_masks, train_labels, Full_LSTM_Model, device, n_epochs = 10, embed_dim = embed_dim, bidi = True, attention = True)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))




running split 1:

LSTM with bidirectional = True, attention = True


 10%|█         | 1/10 [00:01<00:13,  1.48s/it]

Loss: 6.783921897411346


 20%|██        | 2/10 [00:02<00:11,  1.48s/it]

...

Loss: 2.2106542140245438


 90%|█████████ | 9/10 [00:13<00:01,  1.49s/it]

Loss: 1.501184195280075


100%|██████████| 10/10 [00:14<00:00,  1.49s/it]

Loss: 1.201873518526554
0.864876925945282
Final Evaluation for model 10
Accuracy: 0.75
Precision, Recall, F1: (0.5555555555555556, 0.47619047619047616, 0.5128205128205129, None)

===Aggregate Stats===
Accuracy: 0.75
Precision, Recall, F1: (0.5658536585365853, 0.5345622119815668, 0.5497630331753554, None)
